<img src='./1.png'>

# imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import warnings; warnings.filterwarnings(action='ignore')

import matplotlib as mpl
# import statsmodels.api as sm
# import scipy.stats as stats
# import statsmodels.api as sm

from IPython.core.display import display, HTML
# display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)

plt.rcParams['font.family'] = 'Malgun Gothic'

In [2]:
from sklearn import datasets

# preprocessing.encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder # pd.get_dummies
# preprocessing.scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler


# decomposition.PCA
from sklearn.decomposition import PCA

# estimators.regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.linear_model import ElasticNet, ElasticNetCV
# --------------------------------------------------

# neural network -----------------------------------
import tensorflow as tf
import keras
# --------------------------------------------------

# ensemble -----------------------------------------
# ensemble.bagging
from sklearn.ensemble import BaggingClassifier, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
# ensemble.boosting
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
# ensemble.boosting & bagging
from xgboost import XGBRFClassifier, XGBRFRegressor
# ensemble.voting
from sklearn.ensemble import VotingClassifier, VotingRegressor
# --------------------------------------------------

# metrics ------------------------------------------

# metrics.errors
from sklearn.metrics import mean_squared_error     # 'neg_mean_squared_error', 'neg_root_mean_squared_error'
from sklearn.metrics import mean_squared_log_error # 'neg_mean_squared_log_error'
from sklearn.metrics import mean_absolute_error    # 'neg_mean_absolute_error'

# model_selection ----------------------------------
# model_selection.splitter
from sklearn.model_selection import train_test_split
# model_selection.model_validation
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
# model_selection.hyper_parameter_optimizer
from sklearn.model_selection import GridSearchCV
# --------------------------------------------------

# read

In [3]:
train = pd.read_csv("FIFA_train.csv")

test = pd.read_csv("FIFA_test.csv")
sub = pd.read_csv("submission.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'FIFA_train.csv'

In [ ]:
print(train.shape, test.shape)

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
train["continent"].value_counts()

In [ ]:
train["position"].value_counts()

In [ ]:
train["prefer_foot"].value_counts()


In [ ]:
train["contract_until"].value_counts()

In [ ]:
df = pd.concat([train, test])
df.reset_index
df.shape

In [ ]:
train.hist(figsize=(20,15))
plt.show()

In [ ]:
test.hist(figsize=(20,15))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))     
sns.heatmap(train.corr(), annot=True, fmt=".4f", cmap="Blues")

# EDA

In [ ]:
df.head()

In [ ]:
df[df["stat_overall"] > df["stat_potential"]]

In [ ]:
df["성장률"] = df["stat_overall"] /  df["stat_potential"]

In [ ]:
df.head()

In [ ]:
df["contract_until"].unique()

In [ ]:
dict1 = {'2021':2021, '2020':2020, '2019':2019, '2023':2023, '2022':2022, '2024':2024, 'Jun 30, 2019':2019,
       '2026':2026, 'Dec 31, 2018':2018, '2018':2018, '2025':2025, 'Jun 30, 2020':2020,
       'May 31, 2020':2020, 'May 31, 2019':2019, 'Jan 31, 2019':2019, 'Jan 1, 2019':2019,
       'Jan 12, 2019':2019, 'Dec 31, 2019':2019, 'Jun 1, 2019':2019}

In [ ]:
df["contract_until"]=df["contract_until"].map(dict1)

In [ ]:
df["contract_until"].value_counts()

In [ ]:
df["남은계약년도"] = df["contract_until"] - 2018

In [ ]:
df = df.drop(["id", "name"], axis=1)

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df)
df.head()

In [ ]:
train = df.iloc[:train.shape[0]]
test = df.iloc[train.shape[0]:]
print(train.shape, test.shape)

In [ ]:
scale = MinMaxScaler()
scale.fit(np.array(train["contract_until"]).reshape(-1,1))
train["contract_until"] = scale.transform(np.array(train["contract_until"]).reshape(-1,1))
test["contract_until"] = scale.transform(np.array(test["contract_until"]).reshape(-1,1))

In [ ]:
test = test.drop("value", axis=1)


In [ ]:
y = train["value"]
X = train.drop("value", axis=1)

In [ ]:
log_y = np.log1p(y)

In [ ]:
boosting_model = GradientBoostingRegressor(loss='ls', learning_rate=0.1, n_estimators=1000,
                 subsample=1.0, criterion='friedman_mse',  #{'friedman_mse', 'mse', 'mae'}
                          random_state=1414, alpha=0.9)
            
bagging_model = BaggingRegressor(base_estimator=boosting_model,  # None = DecisionTreeRegressor.
                 n_estimators=1000,
                 max_samples=1.0,
                 max_features=1.0,
                 bootstrap=True,
                 oob_score=False,
                 random_state=1414)
bagging_model.fit(X, log_y)
pred = bagging_model.predict(test)

In [ ]:
sub["value"] = np.expm1(pred)
sub.head()

In [ ]:
sub.to_csv("./sub/sub01.csv", index=False)

<img src="./캡처.png">